# メモ書き
## key(項目)
### jpcrp_cor:CompanyNameCoverPage
企業名
### jppfs_cor:Assets
総資産
### jpcrp_cor:NetSalesSummaryOfBusinessResults
売上高:製品や商品など有形物を販売する場合
### jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults
営業収益:無形のサービスや手数料などを提供した場合
### 例外(jpcrp030000-asr_E\*\*\*\*\*-***:NAME)
定義済みではない個社毎に独自に開示しているデータ，E\*\*\*\*\* は会社番号でNAMEに項目名が入る．  
asrはPublicDoc内の.xbrlファイル名に含まれる謎の文字列

## context_ref(詳細)
### FilingDateInstant
企業名のコンテキスト(詳細不明)
### CurrentYearInstant_NonConsolidatedMember
CurrentYearInstant:今年度  
NonConsolidatedMember:単体情報(親会社-持ち株会社のみ) ，ない場合は連結情報（子会社含む，ホールディングスと名前につくことが多い）  
Instantは総資産のみに使われていると思われる
#### CurrentYearDuration_NonConsolidatedMember
Duration:期間 は 代n期を示す  
Current:今期, Prior1:前期, Prior2:前々期, 以降さかのぼるごとにPriorに続く数値が増える

In [35]:
#!usr/bin/env python3
# -*- coding: utf-8 -*-

In [36]:
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
import fnmatch
import os

In [37]:
# 全xbrlファイルパスを取得する(ディレクトリを含んでしまう)
def find_all_xbrl(directory):
    for root, dirs, files in os.walk(directory):
        yield root
        for file in files:
            if fnmatch.fnmatch(file, '*.xbrl'):
                yield os.path.join(root, file)

In [43]:
# 全ファイルパスを取得
all_file_path = []
for f in find_all_xbrl("..\extracted_xbrl"):
    all_file_path.append(f)

In [44]:
# xbrl形式の入るパスのみ取得
xbrl_file_path = []
for i in all_file_path:
    # PublicDoc内のxbrlファイルはasrという文字列を含む
    if fnmatch.fnmatch(i, '*asr*.xbrl'):
        xbrl_file_path.append(i)
print(len(xbrl_file_path))

533


In [45]:
# パーサの初期化
parser = EdinetXbrlParser()

In [46]:
# xbrlファイルを取得してpythonで扱えるオブジェクトに格納
edinet_xbrl_obj = parser.parse_file(xbrl_file_path[0])

In [55]:
# 企業名を取得
print(xbrl_file_path[0])
key = "jpcrp_cor:CompanyNameCoverPage"
context_ref = "FilingDateInstant"
company_name = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Company: " + company_name)

..\extracted_xbrl\Xbrl_Search_20181129_112233\S100D0OG\XBRL\PublicDoc\jpcrp030000-asr-001_E33794-000_2018-03-31_01_2018-06-28.xbrl
Company: 神戸天然物化学株式会社


In [56]:
# 総資産を取得
key = "jppfs_cor:Assets"
context_ref = "CurrentYearInstant_NonConsolidatedMember"
current_year_assets = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Assets: " + current_year_assets + " JPN")

Assets: 12768925000 JPN


In [58]:
# 売上高を取得
key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
context_ref = "CurrentYearDuration_NonConsolidatedMember"
current_year_duration = edinet_xbrl_obj.get_data_by_context_ref(key, context_ref).get_value()
print("Sales: " + current_year_duration + " JPN")

Sales: 6312682000 JPN


In [81]:
# 売上が1000億円以上の企業を取得
target_company = []
target_sales = []

sales_context_ref = "CurrentYearDuration_NonConsolidatedMember" # 単体情報(親会社のみ)

company_key = "jpcrp_cor:CompanyNameCoverPage"
company_context_ref = "FilingDateInstant"

for i in xbrl_file_path:
    sales_key = "jpcrp_cor:NetSalesSummaryOfBusinessResults"
    
    # 指定オブジェクトの取得
    edinet_xbrl_obj = parser.parse_file(i)
    sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高が存在しないなら，営業収益を取得
    if sales is None:
        sales_key = "jpcrp_cor:OperatingRevenue1SummaryOfBusinessResults"
        sales = edinet_xbrl_obj.get_data_by_context_ref(sales_key, sales_context_ref)
    
    # 売上高，営業収益ともにないなら除外
    if sales is None:
        pass
    else:
        sales = sales.get_value()
    
    if sales is None:
        pass
    else:
        # 1000億円以上の売上ならリストに追加
        if int(sales) >= 100000000000:
            company = edinet_xbrl_obj.get_data_by_context_ref(company_key, company_context_ref).get_value()
            target_company.append(company)
            target_sales.append(sales)

print(target_company)
print(target_sales)

['レンゴー株式会社', '日本新薬株式会社', '日本コークス工業株式会社', '株式会社ツムラ', '株式会社カネカ', '宇部興産株式会社', '武田薬品工業株式会社', '日本製紙株式会社', '出光興産株式会社', '信越化学工業株式会社', '関西ペイント株式会社', '大王製紙株式会社', '日立化成株式会社', '出光興産株式会社', '日本コークス工業株式会社', '旭化成株式会社', 'エア・ウォーター株式会社', '積水化学工業株式会社', '日油株式会社', '株式会社エフピコ', '三菱製紙株式会社', '日本化薬株式会社', '北越紀州製紙株式会社', '日産化学工業株式会社', '富士石油株式会社', '東ソー株式会社', '日本ゼオン株式会社', '大日精化工業株式会社', '持田製薬株式会社', '株式会社コーセー', '日本ゼオン株式会社', 'アイカ工業株式会社', '株式会社ＡＤＥＫＡ', '日東電工株式会社', '住友理工株式会社', '東レ株式会社', '三洋化成工業株式会社', 'ダイキョーニシカワ株式会社', '田辺三菱製薬株式会社', '小野薬品工業株式会社', '株式会社トクヤマ', '株式会社ダイセル', 'ハリマ化成グループ株式会社', '三菱瓦斯化学株式会社', '東洋紡株式会社', 'グンゼ株式会社', '参天製薬株式会社', '沢井製薬株式会社', '三井化学株式会社', '小林製薬株式会社', '東洋ゴム工業株式会社', '横浜ゴム株式会社', 'ＤＩＣ株式会社', '昭和電工株式会社', 'ライオン株式会社', '住友ゴム工業株式会社', 'コクヨ株式会社', '小林製薬株式会社', '久光製薬株式会社', '大日本住友製薬株式会社', '株式会社日本触媒', '日医工株式会社', 'トヨタ紡織株式会社', '第一三共株式会社', 'アステラス製薬株式会社', '住友化学株式会社', 'イビデン株式会社', '大陽日酸株式会社', 'ＪＳＲ株式会社', 'コニカミノルタ株式会社', '豊田合成株式会社', 'エーザイ株式会社', '帝人株式会社', '塩野義製薬株式会社', 'デンカ株式会社', 'アステラス製薬株式会社', '関西ペイント株式会社', '大王製紙株式会社', '信越化学工業株式会社'